In [7]:
from yahoo_fin.stock_info import *
from yahoo_fin.options import *
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import random
import math

In [8]:
def get_options_data(list_of_tickers):

    # Building the analysis around the probability of trading weekly contracts
    expiration = get_expiration_dates(list_of_tickers[0])[0]

    options_df = pd.DataFrame()
    for ticker in list_of_tickers:
        try:
            price = get_live_price(ticker)
            option = get_calls(ticker, expiration)
            option['current_price'] = price
            option['ticker'] = ticker
            option['date_exp'] = expiration
            options_df = options_df.append(option)
        except:            
            print('Option contract not available on ' + str(expiration))
            pass

    # Rename the columns to follow PEP8
    options_df = options_df.rename(columns={'Contract Name': 'contract_name', 'Strike': 'strike', 'Bid': 'bid',
                                            'Ask': 'ask', 'Volume': 'vol', 'Open Interest': 'open_interest'})
    # Filter down the columns
    options_df = options_df[['contract_name', 'strike', 'bid', 'ask', 'current_price',
                             'vol', 'open_interest', 'ticker', 'date_exp']]

    options_df['avg_price'] = (options_df['bid'] + options_df['ask']) / 2

    # Creating columns for total option cost and break even percentage
    options_df['option_cost'] = options_df['avg_price'] * 100
    options_df['be_percent'] = ((options_df['strike'] + options_df['avg_price']) / options_df['current_price']) - 1
    options_df['be_percent'] = options_df['be_percent'] * 100
    options_df['be_percent'] = options_df['be_percent'].round(2)
    options_df['be_price'] = options_df['strike'] + options_df['avg_price']

    return options_df

In [9]:
def get_mid_values(x):
    
    """ This functions returns a new DF with the 5 closest values below & above the strike for each ticker"""
    
    lower_bound = x[x['strike'].round() >= x['current_price'].round()]
    upper_bound = x[x['strike'].round() < x['current_price'].round()]
    
    x = pd.concat([lower_bound[:5], upper_bound[-5:]])
    x = x.sort_values('strike')

    return x

In [10]:
tickers = ['TSLA', 'AAPL', 'FB']
df = get_options_data(tickers)
df = df.groupby(['ticker']).apply(lambda x: get_mid_values(x)).reset_index(drop=True)

In [13]:
df.loc[df['ticker'] == 'AAPL']

,contract_name,strike,bid,ask,current_price,vol,open_interest,ticker,date_exp,avg_price,option_cost,BE_percent,BE_price
0,AAPL201106C00110000,110.0,5.00,5.10,114.925003,13076,20755,AAPL,"November 6, 2020",5.050,505.0,0.11,115.050
1,AAPL201106C00111000,111.0,4.15,4.20,114.925003,9108,10363,AAPL,"November 6, 2020",4.175,417.5,0.22,115.175
2,AAPL201106C00112000,112.0,3.35,3.45,114.925003,12669,16650,AAPL,"November 6, 2020",3.400,340.0,0.41,115.400
3,AAPL201106C00113000,113.0,2.54,2.58,114.925003,15839,10261,AAPL,"November 6, 2020",2.560,256.0,0.55,115.560
4,AAPL201106C00114000,114.0,1.97,2.00,114.925003,24910,10985,AAPL,"November 6, 2020",1.985,198.5,0.92,115.985
5,AAPL201106C00115000,115.0,1.36,1.38,114.925003,77117,35031,AAPL,"November 6, 2020",1.370,137.0,1.26,116.370
6,AAPL201106C00116000,116.0,0.94,0.95,114.925003,54761,11774,AAPL,"November 6, 2020",0.945,94.5,1.76,116.945
7,AAPL201106C00117000,117.0,0.62,0.63,114.925003,23515,12270,AAPL,"November 6, 2020",0.625,62.5,2.35,117.625
8,AAPL201106C00118000,118.0,0.39,0.40,114.925003,27533,20212,AAPL,"November 6, 2020",0.395,39.5,3.02,118.395
9,AAPL201106C00119000,119.0,0.25,0.26,114.925003,7635,9093,AAPL,"November 6, 2020",0.255,25.5,3.77,119.255
